# This notebook defines Level 1 sub-macro regions based on the L0 macro regions.
It follows the same flow as L0: load the Admin 0_all_layers dataset,
prepare names/codes, define L1 sub-macro regions, validate coverage,
and plot results.


## 1. Load data


In [5]:
from pathlib import Path
import geopandas as gpd

base_dir = Path.cwd().parents[0]
data_dir = base_dir / "Data" / "World Bank Official Boundaries - Admin 0_all_layers"
shp_path = data_dir / "WB_GAD_ADM0_complete_H2RES.shp"
gdf = gpd.read_file(shp_path)

## 2. Name/ISO preparation (mirror L0 if needed)


In [ ]:
# Reuse any custom ISO/name fixes from L0 if L1 needs the same baseline.
# Example: gdf["ISO_A3_CUSTOM"] = gdf["ISO_A3"]



## 3. Define L1 sub-macro regions (LAC)


In [6]:
# SOUTH AMERICA / CARIBBEAN (must use these names)
SOUTH_AMERICA = [
    'Anguilla (U.K.)','Aruba (Neth.)','Antigua and Barbuda','Argentina',
    'Barbados','Belize','Bermuda (U.K.)','Bonaire, Sint Eustatius and Saba (Neth.)',
    'British Virgin Islands (U.K.)','Cayman Islands (U.K.)',
    'Bolivia','Brazil','Curacao (Neth.)','Cuba','Chile','Colombia','Costa Rica','Dominica',
    'Dominican Republic','Ecuador','El Salvador','Grenada','Guatemala',
    'Guyana','Haiti','Honduras','Jamaica','Nicaragua','Panama',
    'Paraguay','Peru','Republica Bolivariana de Venezuela','Saint Kitts and Nevis',
    'Saint Lucia','Saint Vincent and the Grenadines','Suriname','The Bahamas',
    'Trinidad and Tobago','Uruguay','Falkland Islands','Montserrat (U.K.)',
    'Puerto Rico (U.S.)','Saint-Barthelemy','Saint-Martin (Fr.)',
    'Sint Maarten (Neth.)','Turks and Caicos Islands (U.K.)','United States Virgin Islands (U.S.)',
    'French Guyana','Guadalupe (Fr.)','Martinique (Fr.)'
]

# LAC subregions (aligned to SOUTH_AMERICA names)
LAC_REGIONS = {
    'Hydrocore_South': [
        'Brazil','Uruguay','Paraguay'
    ],
    'Southern_Cone': [
        'Argentina','Chile','Falkland Islands'
    ],
    'Andean_Region': [
        'Colombia','Peru','Ecuador','Bolivia'
    ],
    'Central_America': [
        'Belize','Guatemala','Honduras','El Salvador',
        'Nicaragua','Costa Rica','Panama'
    ],
    'Caribbean': [
        'Cuba','Haiti','Dominican Republic','Jamaica','Trinidad and Tobago',
        'The Bahamas','Barbados','Grenada','Dominica',
        'Antigua and Barbuda','Saint Kitts and Nevis','Saint Lucia',
        'Saint Vincent and the Grenadines',
        'Anguilla (U.K.)','Aruba (Neth.)','Bermuda (U.K.)','British Virgin Islands (U.K.)',
        'Cayman Islands (U.K.)','Curacao (Neth.)','Montserrat (U.K.)','Puerto Rico (U.S.)',
        'Saint-Barthelemy','Saint-Martin (Fr.)','Sint Maarten (Neth.)',
        'Turks and Caicos Islands (U.K.)','United States Virgin Islands (U.S.)',
        'Republica Bolivariana de Venezuela','Suriname','Guyana','French Guyana',
        'Guadalupe (Fr.)','Martinique (Fr.)','Bonaire, Sint Eustatius and Saba (Neth.)'
    ]
}


## 4. Validate coverage and mismatches


In [7]:
# Check that LAC names match the dataset.
lac_set = set(SOUTH_AMERICA)
nam0_set = set(gdf['NAM_0'].unique())
missing_in_data = sorted(lac_set - nam0_set)
print('LAC names not found in NAM_0:', missing_in_data)

# Validate subregion coverage.
subregion_members = {c for countries in LAC_REGIONS.values() for c in countries}
missing_in_subregions = sorted(lac_set - subregion_members)
extra_in_subregions = sorted(subregion_members - lac_set)
print('LAC names not assigned to subregions:', missing_in_subregions)
print('Names in subregions but not in SOUTH_AMERICA:', extra_in_subregions)


LAC names not found in NAM_0: ['Saint-Barthelemy']
LAC names not assigned to subregions: []
Names in subregions but not in SOUTH_AMERICA: []


## 5. Plot and save map(s)


In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt

lac_gdf = gdf[gdf['NAM_0'].isin(SOUTH_AMERICA)].copy()
lac_gdf['LAC_Region'] = None
for region, countries in LAC_REGIONS.items():
    lac_gdf.loc[lac_gdf['NAM_0'].isin(countries), 'LAC_Region'] = region

# Simplify geometry to keep the PDF light.
lac_plot = lac_gdf.copy()
lac_plot['geometry'] = lac_plot['geometry'].simplify(0.05, preserve_topology=True)

out_dir = base_dir / 'Maps' / 'Layer1'
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / 'L1_LAC.pdf'

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
lac_plot.plot(column='LAC_Region', ax=ax, legend=True, linewidth=0.2, edgecolor='white')
ax.set_axis_off()
fig.savefig(out_path, dpi=150, bbox_inches='tight')
out_path
